# Armory Evaluation Declarations, Composition, and Modification

There are data declarations in charmory.blocks that recapitulate standard evaluations
from the armory package. 

In [1]:
import charmory.blocks.cifar10

baseline = charmory.blocks.cifar10.baseline

The `baseline` evaluation is a composite dataclass with some metadata fields
describing the evaluation:

In [2]:
print(f"{type(baseline)=}")
print(f"{baseline.name=}\n{baseline.description=}\n{baseline.author=}")

type(baseline)=<class 'charmory.evaluation.Evaluation'>
baseline.name='cifar_baseline'
baseline.description='Baseline cifar10 image classification'
baseline.author='msw@example.com'


The charmory `Evaluation` class was called "Experiment" in prior versions of the
JATIC Armory library, but was renamed to avoid confusion with the MLflow conception
of Experiment, which is a collection of runs.

Along with the metadata, an `Evaluation` contains some required components. The
`dataset` is a `Dataset` object, which specifies an Armory dataset and a pair
of necessary parameters`:

In [3]:
print(f"{baseline.dataset=}")

baseline.dataset=Dataset(function='armory.data.datasets:cifar10', framework='numpy', batch_size=64)


 
object, the `function` parameter is shown first while details come afterwards; this allows quick visibility of "this is a cifar10 dataset" from the `__str__` representation of the object.
An `Evaluation` also requires a `Model` and `Scenario`

In [4]:
print(f"{baseline.model=}")
print(f"{baseline.scenario=}")

baseline.model=Model(function='armory.baseline_models.pytorch.cifar:get_art_model', model_kwargs={}, wrapper_kwargs={}, weights_file=None, fit=True, fit_kwargs={'nb_epochs': 20})
baseline.scenario=Scenario(function='armory.scenarios.image_classification:ImageClassificationTask', kwargs={})


Here we are using a prefab ART model and the standard Armory Image Classification Task
scenario.

Because this is a "baseline" evaluation, it includes no defense, but does use a PGD
attack to calculate adversarial results.

The optional `Metric` field tells Armory that we want to record additional metrics for
this evaluation. In this case, we are interested in the accuracy of the model on
adversarial examples.

In [5]:
print(f"{baseline.attack=}")
print(f"{baseline.defense=}")
print(f"{baseline.metric=}")

baseline.attack=Attack(function='art.attacks.evasion:ProjectedGradientDescent', kwargs={'batch_size': 1, 'eps': 0.031, 'eps_step': 0.007, 'max_iter': 20, 'num_random_init': 1, 'random_eps': False, 'targeted': False, 'verbose': False}, knowledge='white', use_label=True, type=None)
baseline.defense=None
baseline.metric=Metric(profiler_type='basic', supported_metrics=['accuracy'], perturbation=['linf'], task=['categorical_accuracy'], means=True, record_metric_per_sample=False)


The `charmory.blocks` module is intended as a convenient parts cabinet that allows
users to quickly assemble evaluations using standard components. They behave as standard
Python objects, so you can alter the canned definitions:

In [6]:
baseline.metric = None

If you'd prefer to not have additional tracking. Also, the sub-components of an evaluation
are themselves objects, to be composed at user discretion:

In [10]:
print(f"{charmory.blocks.cifar10.metric=}")

# let's put the metric back into baseline
baseline.metric = charmory.blocks.cifar10.metric

charmory.blocks.cifar10.metric=Metric(profiler_type='basic', supported_metrics=['accuracy'], perturbation=['linf'], task=['categorical_accuracy'], means=True, record_metric_per_sample=False)


Instantiation of the `Engine` class using the `Evaluation` object in `baseline` is
straightforward:

In [11]:
import charmory.engine

engine = charmory.engine.Engine(baseline)

In [12]:
result = engine.run()

2023-04-04 11:13:30 2m38s INFO     art.estimators.classification.pytorch:get_layers:1196 Inferred 4 hidden layers on PyTorch classifier.


Evaluation:   0%|          | 0/157 [00:00<?, ?it/s]

2023-04-04 11:14:22 3m30s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent:generate:199 Creating adversarial samples.
2023-04-04 11:14:26 3m34s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent_pytorch:generate:237 Success rate of attack: 73.44%


Evaluation:   1%|          | 1/157 [00:03<08:42,  3.35s/it]

2023-04-04 11:14:26 3m34s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent:generate:199 Creating adversarial samples.
2023-04-04 11:14:29 3m37s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent_pytorch:generate:237 Success rate of attack: 79.69%


...

2023-04-04 11:23:27 12m35s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent:generate:199 Creating adversarial samples.
2023-04-04 11:23:30 12m38s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent_pytorch:generate:237 Success rate of attack: 68.75%


Evaluation:  99%|█████████▉| 156/157 [09:08<00:03,  3.21s/it]

2023-04-04 11:23:30 12m38s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent:generate:199 Creating adversarial samples.
2023-04-04 11:23:31 12m39s INFO     art.attacks.evasion.projected_gradient_descent.projected_gradient_descent_pytorch:generate:237 Success rate of attack: 68.75%


Evaluation: 100%|██████████| 157/157 [09:08<00:00,  3.50s/it]

2023-04-04 11:23:31 12m39s METRIC   armory.instrument.instrument:_write:743 benign_mean_categorical_accuracy on benign examples w.r.t. ground truth labels: 0.539
2023-04-04 11:23:31 12m39s METRIC   armory.instrument.instrument:_write:743 adversarial_mean_categorical_accuracy on adversarial examples w.r.t. ground truth labels: 0.0002


In [14]:
from pprint import pprint

pprint(result)

{'armory_version': '23.3.0.post4+gcd970d9e',
 'config': {'attack': {'function': 'art.attacks.evasion:ProjectedGradientDescent',
                       'knowledge': 'white',
                       'kwargs': {'batch_size': 1,
                                  'eps': 0.031,
                                  'eps_step': 0.007,
                                  'max_iter': 20,
                                  'num_random_init': 1,
                                  'random_eps': False,
                                  'targeted': False,
                                  'verbose': False},
                       'type': None,
                       'use_label': True},
            'author': 'msw@example.com',
            'dataset': {'batch_size': 64,
                        'framework': 'numpy',
                        'function': 'armory.data.datasets:cifar10'},
            'defense': None,
            'description': 'Baseline cifar10 image classification',
            'metric': {'means': T

# Recap

There is a bunch of explanation and debug prints in this notebook, but the
working code used is quite short:

In [ ]:
import charmory.blocks.cifar10
import charmory.engine

baseline = charmory.blocks.cifar10.baseline
engine = charmory.engine.Engine(baseline)
result = engine.run()